<a href="https://colab.research.google.com/github/Chandini0209/IDS-using-ML-algorithms/blob/main/Capstone_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install gspread oauth2client pandas numpy


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CAPSTONE/creds.json", scope)
client = gspread.authorize(creds)


In [14]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta


In [15]:
!pip install phonenumbers

In [16]:
import phonenumbers

def generate_phone(country, region_mapping):
    # Try to get the country code from the region_mapping first
    code = None
    for region, data in region_mapping.items():
        if country in data["countries"]:
            code = data["dial_code"].get(country)
            break

    if code:
        try:
            # Generate a random 10-digit number
            local_number = ''.join([str(random.randint(0, 9)) for _ in range(10)])
            # Attempt to parse and format using phonenumbers
            # Note: This might not create *valid* numbers, but will follow the format
            # We need a country code without the '+' for the parse function
            country_code_for_parse = code.lstrip('+')
            # We need to find a region code (like US, GB, IN) for the parse function
            # This is a simplified approach; a more robust solution would map countries to region codes
            country_region_code = ""
            for region_key, data in region_mapping.items():
                if country in data.get("countries", []):
                     # This is a simplification; need a proper country to region code mapping
                     # For now, we can try to infer from common country codes or use a default
                     # A more accurate approach would involve a comprehensive country code to region code mapping
                     # Let's use a placeholder logic or rely on the dial code if a direct mapping is complex
                     pass # Will refine this

            # A better approach might be to directly format a random number with the known dial code
            # We'll stick to generating a random 10-digit number and prefixing the dial code
            # as phonenumbers.parse requires a known region or a leading '+' which we already have

            # Let's refine the phonenumbers usage or simplify if direct generation is sufficient
            # Given the user wants the format "+[country_code]-[10_digits]", direct generation is simpler
            number = ''.join([str(random.randint(0, 9)) for _ in range(10)])
            return f"{code}-{number}"

        except Exception as e:
            print(f"Error generating phone for {country} with code {code}: {e}")
            # Fallback to a generic format if phonenumbers fails
            return f"{code}-{random.randint(1000000000, 9999999999)}"
    else:
        # Fallback for countries not in region_mapping
        return f"+00-{random.randint(1000000000, 9999999999)}"

# Now, modify the function call in the lead generation loop in cell 9212e2f3
# (This modification will be done in the next modify_cells command)

In [17]:
#LEADS
import pandas as pd
import random
import uuid
from datetime import datetime, timedelta

# ------------------------------------------
# 1. Region, Country, Dial Code Mapping (Used for phone number generation)
# ------------------------------------------
region_mapping = {
    "NA": {"countries": ["USA", "Canada"], "dial_code": {"USA": "+1", "Canada": "+1"}, "industries": ["Technology", "Healthcare", "Finance", "Retail"], "services": ["Data", "Marketing", "Cloud"]},
    "EU": {"countries": ["UK", "Germany", "France", "Netherlands", "Switzerland", "Sweden", "Israel"], "dial_code": {"UK": "+44", "Germany": "+49", "France": "+33", "Netherlands": "+31", "Switzerland": "+41", "Sweden": "+46", "Israel": "+972"}, "industries": ["Automotive", "Pharmaceuticals", "Banking", "Consulting"], "services": ["Consulting", "Engineering", "Operations"]},
    "APAC": {"countries": ["India", "Japan", "Singapore", "China", "Indonesia", "Australia", "New Zealand", "S. Korea"], "dial_code": {"India": "+91", "Japan": "+81", "Singapore": "+65", "China": "+86", "Indonesia": "+62", "Australia": "+61", "New Zealand": "+64", "S. Korea": "+82"}, "industries": ["Technology", "E-Commerce", "Manufacturing", "Finance"], "services": ["AI", "Staffing", "T&M"]},
    "ME": {"countries": ["UAE", "Saudi Arabia", "Qatar", "Bahrain"], "dial_code": {"UAE": "+971", "Saudi Arabia": "+966", "Qatar": "+974", "Bahrain": "+973"}, "industries": ["Energy", "Real Estate", "Banking"], "services": ["Data", "Consulting"]},
    "LA": {"countries": ["Brazil", "Mexico", "Argentina", "Chile", "Colombia", "Peru"], "dial_code": {"Brazil": "+55", "Mexico": "+52", "Argentina": "+54", "Chile": "+56", "Colombia": "+57", "Peru": "+51"}, "industries": ["Manufacturing", "Agriculture", "Energy"], "services": ["Operations", "Logistics"]},
    "AF": {"countries": ["S. Africa", "Mozambique", "Kenya", "Morocco"], "dial_code": {"S. Africa": "+27", "Mozambique": "+258", "Kenya": "+254", "Morocco": "+212"}, "industries": ["Telecommunications", "Mining", "Finance"], "services": ["Staffing", "Service"]}
}

# ------------------------------------------
# 2. Predefined Lead Data (for Company_Name, Country, Region, Lead_ID) - 108 Companies
# ------------------------------------------
predefined_leads = [
    # North America (20)
    ("Uber", "USA", "NA", "LDUBR-NA-001"), ("Walmart", "USA", "NA", "LDWMT-NA-002"), ("Ford Motor Co", "USA", "NA", "LDFRD-NA-003"), ("Amazon", "USA", "NA", "LDAMZ-NA-004"),
    ("Microsoft", "USA", "NA", "LDMST-NA-005"), ("Target", "USA", "NA", "LDTGT-NA-006"), ("Uber Eats", "USA", "NA", "LDUBE-NA-007"), ("CVS Health", "USA", "NA", "LDCVS-NA-008"),
    ("McKesson", "USA", "NA", "LDMCK-NA-009"), ("TD Bank Group", "Canada", "NA", "LDTDB-NA-010"), ("Shopify", "Canada", "NA", "LDSHF-NA-011"), ("Suncor Energy", "Canada", "NA", "LDSCR-NA-012"),
    ("Kaiser Permanente", "USA", "NA", "LDKPR-NA-013"), ("Wells Fargo", "USA", "NA", "LDWLF-NA-014"), ("Canadian Tire", "Canada", "NA", "LDCTC-NA-015"), ("American Tower", "USA", "NA", "LDAMT-NA-016"),
    ("Tim Hortons", "Canada", "NA", "LDTHC-NA-017"), ("Lowe's", "USA", "NA", "LDLWS-NA-018"), ("PicnicHealth", "USA", "NA", "LDPCH-NA-019"), ("Credit Saison India (USA operations)", "USA", "NA", "LDCSI-NA-020"),

    # Europe (16)
    ("Tesco", "UK", "EU", "LDTSCO-EU-021"), ("Lufthansa Group", "Germany", "EU", "LDLFT-EU-022"), ("Allianz Direct", "Germany", "EU", "LDALZ-EU-023"), ("HSBC", "UK", "EU", "LDHSBC-EU-024"),
    ("Sage Group", "UK", "EU", "LDSGE-EU-025"), ("Schneider Electric", "France", "EU", "LDSNE-EU-026"), ("Philips Healthcare", "Netherlands", "EU", "LDPHL-EU-027"), ("Vodafone", "UK", "EU", "LDVDF-EU-028"),
    ("Siemens", "Germany", "EU", "LDSMN-EU-029"), ("Nestlé", "Switzerland", "EU", "LDNST-EU-030"), ("ING Group", "Netherlands", "EU", "LDING-EU-031"), ("ABB", "Switzerland", "EU", "LDABB-EU-032"),
    ("Volvo Group", "Sweden", "EU", "LDVLV-EU-033"), ("Carrefour", "France", "EU", "LDCFR-EU-034"), ("ZF Group", "Germany", "EU", "LDZFG-EU-035"), ("Trigo", "UK", "EU", "LDTRG-EU-036"), # Assuming UK operations for Trigo

    # Asia-Pacific (14)
    ("DBS Bank", "Singapore", "APAC", "LDDBS-APAC-037"), ("Kiwibank", "New Zealand", "APAC", "LDKWB-APAC-038"), ("Reliance Retail", "India", "APAC", "LDRLR-APAC-039"), ("Tata Consultancy Services", "India", "APAC", "LDTCS-APAC-040"),
    ("SoftBank", "Japan", "APAC", "LDSFB-APAC-041"), ("Tokopedia / GoTo", "Indonesia", "APAC", "LDTKP-APAC-042"), ("Telstra", "Australia", "APAC", "LDTSL-APAC-043"), ("Westpac", "Australia", "APAC", "LDWSP-APAC-044"),
    ("infosys", "India", "APAC", "LDINF-APAC-045"), ("Grab", "Singapore", "APAC", "LDGRB-APAC-046"), ("JioMart", "India", "APAC", "LDJMR-APAC-047"), ("Toyota", "Japan", "APAC", "LDTYT-APAC-048"),
    ("Sony", "Japan", "APAC", "LDSNY-APAC-049"), ("Samsung", "S. Korea", "APAC", "LDSMS-APAC-050"),

    # Middle East (4)
    ("Saudi Aramco", "Saudi Arabia", "ME", "LDSAR-ME-051"), ("ADNOC", "UAE", "ME", "LDADC-ME-052"), ("QatarEnergy", "Qatar", "ME", "LDQTE-ME-053"), ("Investcorp", "Bahrain", "ME", "LDINV-ME-054"),

    # Latin America (10)
    ("Petrobras", "Brazil", "LA", "LDPBR-LA-055"), ("América Móvil", "Mexico", "LA", "LDAMX-LA-056"), ("MercadoLibre", "Argentina", "LA", "LDMCL-LA-057"), ("Codelco", "Chile", "LA", "LDCDL-LA-058"),
    ("Ecopetrol", "Colombia", "LA", "LDECP-LA-059"), ("Latam Airlines", "Chile", "LA", "LDLTM-LA-060"), ("Natura", "Brazil", "LA", "LDNTR-LA-061"), ("Itau Unibanco", "Brazil", "LA", "LDITU-LA-062"),
    ("Bancolombia", "Colombia", "LA", "LDBCL-LA-063"), ("Grupo Mexico", "Mexico", "LA", "LDGPM-LA-064"),

    # Africa (12)
    ("MTN Group", "S. Africa", "AF", "LDMTN-AF-065"), ("Sasol", "S. Africa", "AF", "LDSAL-AF-066"), ("Standard Bank Group", "S. Africa", "AF", "LDSBG-AF-067"), ("Vodacom", "S. Africa", "AF", "LDVDC-AF-068"),
    ("Safaricom", "Kenya", "AF", "LDSFC-AF-069"), ("KCB Group", "Kenya", "AF", "LDKCB-AF-070"), ("Attijariwafa Bank", "Morocco", "AF", "LDATJ-AF-071"), ("FirstRand", "S. Africa", "AF", "LDFRD-AF-072"),
    ("Absa Group", "S. Africa", "AF", "LDABG-AF-073"), ("Nedbank Group", "S. Africa", "AF", "LDNDB-AF-074"), ("Equity Group Holdings", "Kenya", "AF", "LDEGH-AF-075"), ("Telkom", "S. Africa", "AF", "LDTLK-AF-076"),

    # Adding more predefined companies to reach 108, maintaining the pattern and region assignment
    ("Infosys", "India", "APAC", "LDINF-APAC-077"), ("Wipro", "India", "APAC", "LDWIP-APAC-078"), ("HCL Technologies", "India", "APAC", "LDHCL-APAC-079"),
    ("Tech Mahindra", "India", "APAC", "LDTMH-APAC-080"), ("Larsen & Toubro Infotech", "India", "APAC", "LDLTI-APAC-081"), ("Mindtree", "India", "APAC", "LDMNT-APAC-082"),
    ("Mphasis", "India", "APAC", "LDMPH-APAC-083"), ("L&T Technology Services", "India", "APAC", "LDLTTS-APAC-084"), ("Persistent Systems", "India", "APAC", "LDPST-APAC-085"),
    ("Coforge", "India", "APAC", "LDCFG-APAC-086"), ("Zensar Technologies", "India", "APAC", "LDZNR-APAC-087"), ("Firstsource Solutions", "India", "APAC", "LDFRS-APAC-088"),
    ("ExlService Holdings", "India", "APAC", "LDEXL-APAC-089"), ("Genus Power Infrastructures", "India", "APAC", "LDGPI-APAC-090"), ("Cyient", "India", "APAC", "LDCYI-APAC-091"),

    ("Tata Motors", "India", "APAC", "LDTMO-APAC-092"), ("Mahindra & Mahindra", "India", "APAC", "LDMAM-APAC-093"), ("Maruti Suzuki India", "India", "APAC", "LDMSI-APAC-094"),
    ("Hyundai Motor India", "India", "APAC", "LDHMI-APAC-095"), ("Kia India", "India", "APAC", "LDKIA-APAC-096"), ("Toyota Kirloskar Motor", "India", "APAC", "LDTKM-APAC-097"),
    ("Honda Cars India", "India", "APAC", "LDHCI-APAC-098"), ("Renault India", "India", "APAC", "LDRNI-APAC-099"), ("Nissan Motor India", "India", "APAC", "LDNMI-APAC-100"),

    ("Apple", "USA", "NA", "LDAPL-NA-101"), ("Google", "USA", "NA", "LDGOO-NA-102"), ("Facebook", "USA", "NA", "LDFAB-NA-103"), ("Netflix", "USA", "NA", "LDNFL-NA-104"),
    ("Salesforce", "USA", "NA", "LDSFC-NA-105"), ("Tesla", "USA", "NA", "LDTES-NA-106"), ("IBM", "USA", "NA", "LDIBM-NA-107"), ("Intel", "USA", "NA", "LDINT-NA-108")
]

num_predefined_leads = len(predefined_leads) # Define num_predefined_leads here

# ------------------------------------------
# 3. Generate Sales Reps (Unevenly distributed based on region size)
# ------------------------------------------
rep_names = [
    "Alice Smith", "Bob Johnson", "Charlie Lee", "Diana King", "Edward Brown",
    "Fiona Green", "George White", "Hannah Black", "Ian Gray", "Jill Lewis",
    "Kevin Martin", "Laura Scott", "Mike Turner", "Nina Adams", "Oscar Clark",
    "Paula Young", "Quinn Hall", "Rachel Allen", "Steve Wright", "Tina Baker"
] # 20 names

# Distribute 20 reps unevenly across regions (adjust numbers as needed to total 20)
# Example: NA: 4, EU: 4, APAC: 5, ME: 2, LA: 3, AF: 2
region_rep_distribution = {"NA": 4, "EU": 4, "APAC": 5, "ME": 2, "LA": 3, "AF": 2}

rep_list = []
rep_name_index = 0
for region, count in region_rep_distribution.items():
    for i in range(count):
        full_name = rep_names[rep_name_index]
        initials = "".join([part[0] for part in full_name.split()]).upper()
        rep_id = f"{initials}_{region}"
        rep_list.append({"rep_id": rep_id, "rep_name": full_name, "region": region})
        rep_name_index += 1

# Convert to DataFrame
rep_df = pd.DataFrame(rep_list)

# ------------------------------------------
# 4. Generate Leads DataFrame
# ------------------------------------------
def get_company_email(name):
    # Basic email domain generation
    domain = name.lower().replace(" ", "").replace("&", "").replace(".", "").replace("/", "").replace("(", "").replace(")", "") + ".com"
    return domain

# Use the generate_phone function defined in the previous cell (ad0fa588)

def generate_timestamp(start_days_ago=730): # Up to 2 years ago
    created = datetime.now() - timedelta(days=random.randint(0, start_days_ago), minutes=random.randint(0, 1440), seconds=random.randint(0, 86400))

    # Calculate minimum required duration based on stages
    # This is a placeholder; the actual required duration depends on the final stage,
    # which is assigned later. We will ensure last_updated is after created with a buffer.
    min_buffer_days = 1 # Ensure at least one day difference if a lead progresses

    # Generate last_updated ensuring it's after created with a buffer
    last_updated = created + timedelta(days=random.randint(min_buffer_days, 60), minutes=random.randint(0, 1440), seconds=random.randint(0, 86400)) # Ensure updated is after created with a reasonable range

    return created.strftime("%Y-%m-%d %H:%M:%S"), last_updated.strftime("%Y-%m-%d %H:%M:%S")

def get_company_initials(name):
    return "".join([w[0] for w in name.split()]).upper()

# Define service categories and industries based on region_mapping for random lead generation
all_service_categories = list(set([service for region_data in region_mapping.values() for service in region_data.get('services', [])]))
all_industries = list(set([industry for region_data in region_mapping.values() for industry in region_data.get('industries', [])]))


# Generate a list of more descriptive company name parts for random leads
company_name_parts = {
    "adjectives": ["Global", "Strategic", "Innovative", "Dynamic", "Future", "Leading", "Creative", "Advanced"],
    "nouns": ["Solutions", "Systems", "Analytics", "Consulting", "Tech", "Ventures", "Partners", "Industries"]
}

def generate_random_company_name(adjectives, nouns):
    return f"{random.choice(adjectives)}{random.choice(nouns)}"

def get_abbreviated_company_name(name):
    # Simple abbreviation logic
    words = name.split()
    if len(words) > 1:
        return "".join([w[:2] for w in words]).upper() # Use first two letters of each word
    else:
        return name[:4].upper() # Use first four letters if single word


leads_data = []

# Define stages in the desired funnel order
funnel_stages = [
    "Lead Captured",
    "Qualified",
    "Contacted",
    "Responded",
    "Demo Scheduled",
    "Demo",
    "Demo Completed",
    "Proposal Sent",
    "Proposal",
    "Negotiation"
]

# Define stages where a deal could potentially be lost *before* Negotiation
potential_early_loss_stages = ["Demo", "Demo Completed", "Proposal Sent", "Proposal"]
# Define criteria for losing a deal at earlier stages
early_loss_criteria = {
    "Demo": {"engagement_level": ["Low"], "random_chance": 0.4}, # 40% chance of losing at Demo if engagement is Low
    "Demo Completed": {"budget_tier": ["20-50L"], "engagement_level": ["Low"], "random_chance": 0.3}, # 30% chance after Demo Completed if budget low and engagement low
    "Proposal": {"engagement_level": ["Low", "Medium"], "priority": ["Low"], "random_chance": 0.5} # 50% chance at Proposal if engagement low/medium and priority low
}
# Define criteria for losing a deal after Negotiation
negotiation_loss_criteria = {"budget_tier": ["20-50L"], "engagement_level": ["Low"], "random_chance": 0.6} # 60% chance after Negotiation if budget low and engagement low


# Add predefined leads
for company_name, country, region, lead_id in predefined_leads:
    company_email_domain = get_company_email(company_name)
    company_id = f"CMP_{get_company_initials(company_name)}"

    created_date, last_updated = generate_timestamp()
    deal_source = random.choices(["Inbound", "Outbound"], weights=[0.2, 0.8])[0]
    phone = generate_phone(country, region_mapping)

    region_reps = rep_df[rep_df["region"] == region]
    if not region_reps.empty:
        rep_row = region_reps.sample(n=1).iloc[0]
        assigned_rep_id = rep_row["rep_id"]
    else:
        assigned_rep_id = "Unassigned"

    lead_email = f"contact_{random.randint(1,100)}@{company_email_domain}"

    current_region = None
    for r, data in region_mapping.items():
        if country in data["countries"]:
            current_region = r
            break

    service_category = random.choice(region_mapping.get(current_region, {}).get('services', all_service_categories)) if current_region and region_mapping.get(current_region, {}).get('services') else random.choice(all_service_categories)
    company_industry = random.choice(region_mapping.get(current_region, {}).get('industries', all_industries)) if current_region and region_mapping.get(current_region, {}).get('industries') else random.choice(all_industries)

    # Assign other random data first, as these influence the final stage determination
    engagement_level = random.choice(["High", "Medium", "Low"])
    follow_up_threshold = random.choice([3, 5, 7, 10])
    project_term = random.choice(["0-6 months", "6-12 months", "12-24 months", ">24 months"])
    budget_tier = random.choice(["20-50L", "50-100L", "100L+"])
    priority = random.choice(["High", "Medium", "Low"])

    # Determine the final stage based on sequential progression and 'Deal Lost' conditions
    final_stage = "Lead Captured" # Start at the first stage

    # Progress through stages sequentially, checking for 'Deal Lost' conditions at each potential early loss stage
    for stage in funnel_stages:
        if stage == "Lead Captured":
            pass # Always start at Lead Captured
        elif stage in potential_early_loss_stages:
             # Check if the lead is lost at this stage based on criteria
             met_early_loss_criteria = False
             if stage in early_loss_criteria:
                 criteria = early_loss_criteria[stage]
                 budget_match = budget_tier in criteria.get("budget_tier", [budget_tier])
                 engagement_match = engagement_level in criteria.get("engagement_level", [engagement_level])
                 priority_match = priority in criteria.get("priority", [priority])
                 random_chance_met = random.random() < criteria.get("random_chance", 0)

                 if budget_match and engagement_match and priority_match and random_chance_met:
                     met_early_loss_criteria = True

             if met_early_loss_criteria:
                 final_stage = "Deal Lost"
                 break # Stop progressing if deal is lost

             else:
                 final_stage = stage # If not lost, progress to this stage

        elif stage == "Negotiation":
             final_stage = stage # Progress to Negotiation

        # If we reached Negotiation and haven't been lost, check for 'Deal Won' or 'Deal Lost'
        if final_stage == "Negotiation":
             # Check if the lead is lost after Negotiation based on criteria
             met_negotiation_loss_criteria = False
             criteria = negotiation_loss_criteria
             budget_match = budget_tier in criteria.get("budget_tier", [budget_tier])
             engagement_match = engagement_level in criteria.get("engagement_level", [engagement_level])
             random_chance_met = random.random() < criteria.get("random_chance", 0)

             if budget_match and engagement_match and random_chance_met:
                 final_stage = "Deal Lost"
             else:
                 final_stage = "Deal Won" # If not lost after negotiation, it's a win
             break # Stop progressing after deciding Won/Lost at or after Negotiation

        # If not a potential early loss stage and not Negotiation, just progress
        else:
            final_stage = stage # Progress to the next stage


    leads_data.append({
        "Lead_ID": lead_id,
        "Lead_Name": f"{company_name} Lead",
        "Company_Name": company_name,
        "Source": deal_source,
        "Region_ID": region,
        "Rep_ID": assigned_rep_id,
        "Created_Date": created_date,
        "Current_Stage": final_stage, # Use the determined final stage
        "Email": lead_email,
        "Phone": phone,
        "Priority": priority, # Use the generated priority
        "Last_Updated": last_updated,
        "Deal_Source": deal_source,
        "Company_ID": company_id,
        "Engagement_Level": engagement_level, # Use the generated engagement level
        "Follow_Up_Threshold": follow_up_threshold, # Use the generated follow up threshold
        "Project_Term": project_term, # Use the generated project term
        "Budget_Tier": budget_tier, # Use the generated budget tier
        "Country": country,
        "Company_Industry": company_industry,
        "Service_Category": service_category
    })

num_additional_leads = 5000 - num_predefined_leads

# Get lists of all possible values for random generation
all_countries = [country for region_data in region_mapping.values() for country in region_data["countries"]]
all_regions = list(region_mapping.keys())

for i in range(num_additional_leads):
    company_name = generate_random_company_name(company_name_parts["adjectives"], company_name_parts["nouns"])
    country = random.choice(all_countries)
    region = next(r for r, data in region_mapping.items() if country in data["countries"])

    company_abbreviation = get_abbreviated_company_name(company_name)
    lead_id = f"LD{company_abbreviation}-{region}-{i+1:04}"

    company_email_domain = get_company_email(company_name)
    company_id = f"CMP_{get_company_initials(company_name)}_{i+1:04}"

    created_date, last_updated = generate_timestamp()
    deal_source = random.choices(["Inbound", "Outbound"], weights=[0.2, 0.8])[0]
    phone = generate_phone(country, region_mapping)

    region_reps = rep_df[rep_df["region"] == region]
    if not region_reps.empty:
        rep_row = region_reps.sample(n=1).iloc[0]
        assigned_rep_id = rep_row["rep_id"]
    else:
        assigned_rep_id = "Unassigned"

    lead_email = f"contact_{random.randint(1,100)}@{company_email_domain}"

    current_region = None
    for r, data in region_mapping.items():
        if country in data["countries"]:
            current_region = r
            break

    service_category = random.choice(region_mapping.get(current_region, {}).get('services', all_service_categories)) if current_region and region_mapping.get(current_region, {}).get('services') else random.choice(all_service_categories)
    company_industry = random.choice(region_mapping.get(current_region, {}).get('industries', all_industries)) if current_region and region_mapping.get(current_region, {}).get('industries') else random.choice(all_industries)


    # Assign other random data first, as these influence the final stage determination
    engagement_level = random.choice(["High", "Medium", "Low"])
    follow_up_threshold = random.choice([3, 5, 7, 10])
    project_term = random.choice(["0-6 months", "6-12 months", "12-24 months", ">24 months"])
    budget_tier = random.choice(["20-50L", "50-100L", "100L+"])
    priority = random.choice(["High", "Medium", "Low"])

    # Determine the final stage based on sequential progression and 'Deal Lost' conditions
    final_stage = "Lead Captured" # Start at the first stage

    # Progress through stages sequentially, checking for 'Deal Lost' conditions at each potential early loss stage
    for stage in funnel_stages:
        if stage == "Lead Captured":
            pass # Always start at Lead Captured
        elif stage in potential_early_loss_stages:
             # Check if the lead is lost at this stage based on criteria
             met_early_loss_criteria = False
             if stage in early_loss_criteria:
                 criteria = early_loss_criteria[stage]
                 budget_match = budget_tier in criteria.get("budget_tier", [budget_tier])
                 engagement_match = engagement_level in criteria.get("engagement_level", [engagement_level])
                 priority_match = priority in criteria.get("priority", [priority])
                 random_chance_met = random.random() < criteria.get("random_chance", 0)


                 if budget_match and engagement_match and priority_match and random_chance_met:
                     met_early_loss_criteria = True

             if met_early_loss_criteria:
                 final_stage = "Deal Lost"
                 break # Stop progressing if deal is lost

             else:
                 final_stage = stage # If not lost, progress to this stage

        elif stage == "Negotiation":
             final_stage = stage # Progress to Negotiation

        # If we reached Negotiation and haven't been lost, check for 'Deal Won' or 'Deal Lost'
        if final_stage == "Negotiation":
             # Check if the lead is lost after Negotiation based on criteria
             met_negotiation_loss_criteria = False
             criteria = negotiation_loss_criteria
             budget_match = budget_tier in criteria.get("budget_tier", [budget_tier])
             engagement_match = engagement_level in criteria.get("engagement_level", [engagement_level])
             random_chance_met = random.random() < criteria.get("random_chance", 0)


             if budget_match and engagement_match and random_chance_met:
                 final_stage = "Deal Lost"
             else:
                 final_stage = "Deal Won" # If not lost after negotiation, it's a win
             break # Stop progressing after deciding Won/Lost at or after Negotiation

        # If not a potential early loss stage and not Negotiation, just progress
        else:
            final_stage = stage # Progress to the next stage


    leads_data.append({
        "Lead_ID": lead_id,
        "Lead_Name": f"{company_name} Lead",
        "Company_Name": company_name,
        "Source": deal_source,
        "Region_ID": region,
        "Rep_ID": assigned_rep_id,
        "Created_Date": created_date,
        "Current_Stage": final_stage, # Use the determined final stage
        "Email": lead_email,
        "Phone": phone,
        "Priority": priority, # Use the generated priority
        "Last_Updated": last_updated,
        "Deal_Source": deal_source,
        "Company_ID": company_id,
        "Engagement_Level": engagement_level, # Use the generated engagement level
        "Follow_Up_Threshold": follow_up_threshold, # Use the generated follow up threshold
        "Project_Term": project_term, # Use the generated project term
        "Budget_Tier": budget_tier, # Use the generated budget tier
        "Country": country,
        "Company_Industry": company_industry,
        "Service_Category": service_category
    })


df_lead = pd.DataFrame(leads_data)

df_lead = df_lead[[
    "Lead_ID", "Lead_Name", "Company_Name", "Source", "Region_ID", "Rep_ID",
    "Created_Date", "Current_Stage", "Email", "Phone", "Priority", "Last_Updated",
    "Deal_Source", "Company_ID", "Engagement_Level", "Follow_Up_Threshold",
    "Project_Term", "Budget_Tier", "Country", "Company_Industry", "Service_Category"
]]


# After generating the initial df_lead, update the 'Current_Stage'
# to reflect the latest stage from the generated df_stage_history.
# This part needs to be executed after df_stage_history is generated in the next cell.
# This logic will be moved to a subsequent cell or integrated carefully if possible.


# ------------------------------------------
# 5. Export or View
# ------------------------------------------
# Save to CSV
# df_lead.to_csv("crm_leads_generated.csv", index=False)

# Display sample
display(df_lead.head())
display(df_lead.tail())
print(f"Generated {len(df_lead)} leads.")

,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Last_Updated,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category
0,LDUBR-NA-001,Uber Lead,Uber,Outbound,NA,AS_NA,2025-03-24 11:49:55,Deal Lost,contact_34@uber.com,+1-0099765257,...,2025-05-09 09:57:27,Outbound,CMP_U,Low,7,6-12 months,50-100L,USA,Retail,Cloud
1,LDWMT-NA-002,Walmart Lead,Walmart,Outbound,NA,CL_NA,2025-06-02 21:03:44,Deal Won,contact_11@walmart.com,+1-7090059727,...,2025-07-22 20:06:49,Outbound,CMP_W,Medium,5,0-6 months,100L+,USA,Healthcare,Cloud
2,LDFRD-NA-003,Ford Motor Co Lead,Ford Motor Co,Outbound,NA,BJ_NA,2024-02-20 17:47:39,Deal Won,contact_83@fordmotorco.com,+1-2222484164,...,2024-04-22 00:11:43,Outbound,CMP_FMC,High,7,0-6 months,20-50L,USA,Retail,Data
3,LDAMZ-NA-004,Amazon Lead,Amazon,Inbound,NA,CL_NA,2023-10-09 19:32:02,Deal Won,contact_64@amazon.com,+1-5554699842,...,2023-11-16 01:57:55,Inbound,CMP_A,Medium,3,6-12 months,20-50L,USA,Finance,Data
4,LDMST-NA-005,Microsoft Lead,Microsoft,Outbound,NA,AS_NA,2024-06-12 22:45:26,Deal Lost,contact_27@microsoft.com,+1-0030706612,...,2024-07-03 03:48:00,Outbound,CMP_M,Low,3,0-6 months,100L+,USA,Healthcare,Cloud


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Last_Updated,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category
4995,LDFUTU-EU-4888,FutureSolutions Lead,FutureSolutions,Inbound,EU,EB_EU,2024-10-15 09:50:41,Deal Lost,contact_15@futuresolutions.com,+972-2130522820,...,2024-11-02 05:06:56,Inbound,CMP_F_4888,Medium,3,12-24 months,20-50L,Israel,Automotive,Consulting
4996,LDGLOB-LA-4889,GlobalSolutions Lead,GlobalSolutions,Outbound,LA,PY_LA,2025-06-10 11:27:29,Deal Won,contact_29@globalsolutions.com,+57-2783786847,...,2025-08-02 02:53:34,Outbound,CMP_G_4889,Medium,3,>24 months,20-50L,Colombia,Energy,Logistics
4997,LDCREA-APAC-4890,CreativeVentures Lead,CreativeVentures,Inbound,APAC,IG_APAC,2024-07-08 21:24:55,Deal Won,contact_93@creativeventures.com,+64-3599427694,...,2024-08-25 21:39:37,Inbound,CMP_C_4890,Medium,7,6-12 months,50-100L,New Zealand,Technology,AI
4998,LDCREA-APAC-4891,CreativeConsulting Lead,CreativeConsulting,Outbound,APAC,LS_APAC,2024-05-11 23:29:15,Deal Won,contact_47@creativeconsulting.com,+81-8662752026,...,2024-05-25 13:27:48,Outbound,CMP_C_4891,High,10,6-12 months,50-100L,Japan,Manufacturing,Staffing
4999,LDCREA-ME-4892,CreativeTech Lead,CreativeTech,Outbound,ME,NA_ME,2024-01-14 06:42:25,Deal Won,contact_45@creativetech.com,+966-2739863552,...,2024-01-19 10:15:25,Outbound,CMP_C_4892,High,3,12-24 months,50-100L,Saudi Arabia,Real Estate,Consulting


Generated 5000 leads.


In [18]:
#COMPANIES
import pandas as pd
import numpy as np
import random

industries = ["Technology", "Healthcare", "Finance", "Manufacturing", "Retail", "E-Commerce", "Green Tech", "Mobility", "Real Estate", "Banking", "Logistics", "Housing", "Energy"] # Expanded industries
sizes = ["Start-up", "Small business", "Medium Enterprise", "Large Enterprise"]
ownerships = ["Public Sector", "Private Sector", "Joint Venture", "Franchise"]
funding = ["Bootstrapped", "Seed", "Series A", "Series B", "IPO"]
interest_areas = ["Data", "Marketing", "Cloud", "Design", "HR", "AI", "Consulting", "Engineering", "Operations", "Staffing", "T&M"] # Expanded interest areas

company_data = []
added_company_ids = set()

# Add companies from df_lead
for index, row in df_lead.iterrows():
    company_id = row['Company_ID']
    company_name = row['Company_Name']
    country = row['Country']
    # Add company data only if not already added
    if company_id not in added_company_ids:
        # Attempt to get industry and service from region_mapping if available, otherwise use random
        # Find the country info in the region_mapping dictionary
        current_region_data = next((data for region, data in region_mapping.items() if country in data["countries"]), None)

        company_industry = random.choice(current_region_data.get('industries', industries)) if current_region_data and current_region_data.get('industries') else random.choice(industries)
        interest_area = random.choice(current_region_data.get('services', interest_areas)) if current_region_data and current_region_data.get('services') else random.choice(interest_areas)

        company_data.append({
            "Company_ID": company_id,
            "Company_Name": company_name,
            "Company_Industry": company_industry, # Use industry from region_mapping or random
            "Company_Size": random.choice(sizes),
            "Company_Ownership": random.choice(ownerships),
            "Company_Funding": random.choice(funding),
            "Country": country, # Use country from df_lead
            "Interest_Area": interest_area # Use service from region_mapping or random
        })
        added_company_ids.add(company_id)

# Generate additional companies if needed (optional, based on whether df_lead has all companies)
# If df_lead contains all the companies you need for df_company, you might not need this part.
# Assuming df_lead has the comprehensive list of companies you want in df_company.
# (No additional generation needed as df_lead now contains 10000 companies)

df_company = pd.DataFrame(company_data)
display(df_company.head())
display(df_company.tail())
print(f"Generated {len(df_company)} companies.")

,Company_ID,Company_Name,Company_Industry,Company_Size,Company_Ownership,Company_Funding,Country,Interest_Area
0,CMP_U,Uber,Finance,Large Enterprise,Public Sector,Series B,USA,Marketing
1,CMP_W,Walmart,Healthcare,Start-up,Public Sector,IPO,USA,Marketing
2,CMP_FMC,Ford Motor Co,Healthcare,Large Enterprise,Franchise,IPO,USA,Marketing
3,CMP_A,Amazon,Retail,Large Enterprise,Public Sector,IPO,USA,Marketing
4,CMP_M,Microsoft,Healthcare,Medium Enterprise,Joint Venture,Series A,USA,Cloud


,Company_ID,Company_Name,Company_Industry,Company_Size,Company_Ownership,Company_Funding,Country,Interest_Area
4958,CMP_F_4888,FutureSolutions,Banking,Large Enterprise,Public Sector,Series A,Israel,Operations
4959,CMP_G_4889,GlobalSolutions,Energy,Start-up,Franchise,Bootstrapped,Colombia,Logistics
4960,CMP_C_4890,CreativeVentures,E-Commerce,Small business,Public Sector,Bootstrapped,New Zealand,T&M
4961,CMP_C_4891,CreativeConsulting,Finance,Small business,Joint Venture,Seed,Japan,T&M
4962,CMP_C_4892,CreativeTech,Banking,Start-up,Private Sector,Series A,Saudi Arabia,Consulting


Generated 4963 companies.


In [19]:
#REGION
import pandas as pd
import numpy as np
import random

# Use unique Region_IDs from df_lead
region_ids = df_lead['Region_ID'].unique().tolist()
timezones = ["IST", "EST", "PST", "CET", "GMT"]
countries_in_leads = df_lead['Country'].unique().tolist() # Get countries from df_lead

region_data = []

# Define value mappings for company size for target calculation
company_size_value = {
    "Start-up": 1,
    "Small business": 5,
    "Medium Enterprise": 10,
    "Large Enterprise": 20
}

for region_id in region_ids:
    # Select a country that belongs to this region based on region_mapping
    country = random.choice([c for r, data in region_mapping.items() if r == region_id for c in data['countries'] if c in countries_in_leads])

    # Find reps assigned to this region
    reps_in_region = rep_df[rep_df['region'] == region_id]['rep_name'].tolist()
    region_head = random.choice(reps_in_region) if reps_in_region else "Unassigned"

    # Calculate Region_Target based on companies in the region
    companies_in_region = df_company[df_company['Country'].apply(lambda x: x in region_mapping.get(region_id, {}).get('countries', []))]

    # Calculate a score for each company based on size, ownership, funding, and interest area (simplified)
    # You can adjust weights and factors based on your desired logic
    company_scores = companies_in_region.apply(
        lambda row: company_size_value.get(row['Company_Size'], 0) *
                    (2 if row['Company_Ownership'] == 'Private Sector' else 1) *
                    (2 if row['Company_Funding'] in ['Series B', 'IPO'] else 1) *
                    (2 if row['Interest_Area'] in region_mapping.get(region_id, {}).get('services', []) else 1),
        axis=1
    )

    # Aggregate scores to get a raw region target
    raw_region_target = company_scores.sum()

    # Scale the raw target to a reasonable range (adjust scaling factor as needed)
    # Example: Scale by 100 to get targets in the thousands/millions
    region_target = round(raw_region_target * 500, 2) # Adjusted scaling factor

    region_data.append({
        "Region_ID": region_id,
        "Region_Name": f"Region_{region_id}",
        "Region_Head": region_head,
        "Timezone": random.choice(timezones),
        "Region_Target": region_target
    })

df_region = pd.DataFrame(region_data)
display(df_region.head())

,Region_ID,Region_Name,Region_Head,Timezone,Region_Target
0,NA,Region_NA,Diana King,PST,5893000
1,EU,Region_EU,Fiona Green,PST,16624000
2,APAC,Region_APAC,Laura Scott,IST,22113000
3,ME,Region_ME,Nina Adams,EST,9963000
4,LA,Region_LA,Paula Young,CET,14256000


In [20]:
#SALES_REP
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Use unique Rep_IDs and their corresponding regions from df_lead
sales_rep_data = []
added_rep_ids = set()

for index, row in df_lead.iterrows():
    rep_id = row['Rep_ID']
    region_id = row['Region_ID']
    # Add rep data only if not already added
    if rep_id not in added_rep_ids:
        # Find the rep name from the rep_df based on Rep_ID
        rep_info = rep_df[rep_df['rep_id'] == rep_id]
        rep_name = rep_info['rep_name'].iloc[0] if not rep_info.empty else f"Rep_{rep_id}"


        sales_rep_data.append({
            "Rep_ID": rep_id,
            "Rep_Name": rep_name, # Use name from rep_df
            "Email": f"{rep_id.lower()}@crmcorp.com", # Generate email based on Rep_ID
            "Phone": f"+91-9{random.randint(100000000,999999999)}", # Keep existing phone generation for now
            "Region_ID": region_id,
            "Join_Date": datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1000)),
            "Current_Status": random.choice(["Active", "Inactive", "On Leave"]),
            "Performance_Rating": round(random.uniform(1.0, 5.0), 1),
            "Total_Deals_Closed": random.randint(0, 500),
            "Avg_conversion_time": round(random.uniform(3.0, 30.0), 2),
            "Win_rate": round(random.uniform(0.1, 0.9), 2)
        })
        added_rep_ids.add(rep_id)

# Add any reps from rep_df who are not assigned to any lead in df_lead
reps_not_in_leads = rep_df[~rep_df['rep_id'].isin(added_rep_ids)]
for index, rep_row in reps_not_in_leads.iterrows():
    rep_id = rep_row['rep_id']
    region_id = rep_row['region']
    rep_name = rep_row['rep_name']

    sales_rep_data.append({
        "Rep_ID": rep_id,
        "Rep_Name": rep_name,
        "Email": f"{rep_id.lower()}@crmcorp.com",
        "Phone": f"+91-9{random.randint(100000000,999999999)}",
        "Region_ID": region_id,
        "Join_Date": datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1000)),
        "Current_Status": random.choice(["Active", "Inactive", "On Leave"]),
        "Performance_Rating": round(random.uniform(1.0, 5.0), 1),
        "Total_Deals_Closed": random.randint(0, 500),
        "Avg_conversion_time": round(random.uniform(3.0, 30.0), 2),
        "Win_rate": round(random.uniform(0.1, 0.9), 2)
    })


df_sales_rep = pd.DataFrame(sales_rep_data)
display(df_sales_rep.head())
display(df_sales_rep.tail())
print(f"Generated {len(df_sales_rep)} sales reps.")

,Rep_ID,Rep_Name,Email,Phone,Region_ID,Join_Date,Current_Status,Performance_Rating,Total_Deals_Closed,Avg_conversion_time,Win_rate
0,AS_NA,Alice Smith,as_na@crmcorp.com,+91-9306395858,NA,2020-05-01,Inactive,4.2,115,13.63,0.83
1,CL_NA,Charlie Lee,cl_na@crmcorp.com,+91-9845465486,NA,2021-09-03,On Leave,2.7,343,13.31,0.12
2,BJ_NA,Bob Johnson,bj_na@crmcorp.com,+91-9820730309,NA,2021-03-27,Active,2.2,121,19.08,0.71
3,DK_NA,Diana King,dk_na@crmcorp.com,+91-9290658576,NA,2022-08-21,On Leave,2.2,205,13.91,0.85
4,GW_EU,George White,gw_eu@crmcorp.com,+91-9617212180,EU,2022-07-19,Inactive,2.4,68,24.47,0.66


,Rep_ID,Rep_Name,Email,Phone,Region_ID,Join_Date,Current_Status,Performance_Rating,Total_Deals_Closed,Avg_conversion_time,Win_rate
15,RA_LA,Rachel Allen,ra_la@crmcorp.com,+91-9593653934,LA,2020-10-03,On Leave,3.5,116,3.27,0.49
16,QH_LA,Quinn Hall,qh_la@crmcorp.com,+91-9759983674,LA,2020-02-29,Active,2.3,267,3.77,0.81
17,SW_AF,Steve Wright,sw_af@crmcorp.com,+91-9121507597,AF,2022-02-09,Active,3.0,391,23.41,0.73
18,TB_AF,Tina Baker,tb_af@crmcorp.com,+91-9486145136,AF,2021-12-19,Active,2.9,396,4.83,0.45
19,LS_APAC,Laura Scott,ls_apac@crmcorp.com,+91-9871002851,APAC,2022-07-01,Inactive,3.4,108,14.37,0.68


Generated 20 sales reps.


In [21]:
#STAGE HISTORY
import pandas as pd
import random
from datetime import datetime, timedelta

# Define stages in the desired funnel order based on user provided ranges
funnel_stages = [
    "Lead Captured",
    "Qualified",
    "Contacted",
    "Responded",
    "Demo Scheduled",
    "Demo",
    "Demo Completed",
    "Proposal Sent",
    "Proposal",
    "Negotiation"
]

stage_history_data = []
history_id = 1

# Define realistic time ranges for progression between stages (in days)
# Using the specific ranges provided by the user
stage_time_ranges = {
    "Lead Captured": (2, 7),        # 2–7 days to qualification
    "Qualified": (4, 9),            # 4–9 days to contact
    "Contacted": (5, 12),           # 5–12 days to responded/demo
    "Responded": (3, 6),            # 3–6 days to demo scheduled
    "Demo Scheduled": (4, 8),       # 4–8 days to demo
    "Demo": (2, 3),                 # 2–3 days to complete demo
    "Demo Completed": (4, 9),       # 4–9 days to proposal
    "Proposal Sent": (5, 11),       # 5–11 days to proposal
    "Proposal": (6, 13),            # 6–13 days to negotiation
    "Negotiation": (7, 15)          # 7–15 days to close
}

def add_random_time_of_day(date):
    """Adds a random time of day (hour, minute, second) to a date."""
    # Ensure 'date' is a datetime object before adding timedelta
    if isinstance(date, datetime):
        return date + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
    elif isinstance(date, pd.Timestamp):
         return date + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
    else:
        # If it's a date object, convert to datetime before adding time
        return datetime(date.year, date.month, date.day) + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))


for index, lead in df_lead.iterrows():
    lead_id = lead.Lead_ID
    final_stage = lead['Current_Stage']

    # Determine the sequence of stages the lead went through
    try:
        # If the final stage is Deal Won or Deal Lost, the history should include
        # all stages up to the stage *before* the final outcome (if it was a loss before negotiation)
        # or up to Negotiation if it was a win/loss after Negotiation.
        if final_stage in ["Deal Won", "Deal Lost"]:
             # Find the index of the final stage in the funnel_stages.
             # If the final stage is not in funnel_stages (i.e., Won/Lost),
             # the history goes up to Negotiation (the last stage before Won/Lost).
             try:
                 final_stage_index_in_funnel = funnel_stages.index(final_stage)
                 # If the final stage is in funnel_stages, history goes up to that stage.
                 ordered_stages = funnel_stages[:final_stage_index_in_funnel + 1]
             except ValueError:
                 # If final stage is 'Deal Won' or 'Deal Lost', history goes up to 'Negotiation'
                 if "Negotiation" in funnel_stages:
                     final_history_stage_index = funnel_stages.index("Negotiation")
                     ordered_stages_for_history = funnel_stages[:final_history_stage_index + 1]
                 else:
                     # Fallback if Negotiation is not in funnel_stages
                     ordered_stages_for_history = funnel_stages # Include all stages if structure is different

                 # Add the actual final stage ('Deal Won' or 'Deal Lost') as the last entry in history
                 ordered_stages = ordered_stages_for_history + [final_stage]

        else:
            # If the final stage is an intermediate stage, history goes up to that stage
            final_stage_index = funnel_stages.index(final_stage)
            ordered_stages = funnel_stages[:final_stage_index + 1]

    except ValueError:
        # If the assigned stage is not in funnel_stages or is invalid, default to "Lead Captured"
        print(f"Warning: Lead {lead.Lead_ID} has an invalid final stage '{final_stage}'. Setting history to 'Lead Captured'.")
        ordered_stages = ["Lead Captured"]
        final_stage = "Lead Captured" # Update final_stage variable as well


    start_date_str = lead.Created_Date
    last_updated_date_str = lead.Last_Updated

    # Convert to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M:%S")
    last_updated_date = datetime.strptime(last_updated_date_str, "%Y-%m-%d %H:%M:%S")

    total_duration_seconds = (last_updated_date - start_date).total_seconds()
    num_transitions = len(ordered_stages) - 1

    # Generate timestamps for each stage
    current_timestamp = start_date
    for seq, stage in enumerate(ordered_stages, start=1):
        timestamp = current_timestamp

        # If it's the first stage, use the created date with random time
        if seq == 1:
            timestamp = add_random_time_of_day(start_date)
             # Ensure the generated timestamp is not before the original created_date if original had time
            if timestamp < start_date:
                 timestamp = start_date + timedelta(seconds=random.randint(1, 60)) # Ensure strictly after if on same day


        # If it's the last stage, use the last updated date
        elif seq == len(ordered_stages):
            timestamp = last_updated_date
            # Ensure the last stage timestamp is not before the previous stage timestamp
            if timestamp <= current_timestamp:
                 timestamp = current_timestamp + timedelta(seconds=random.randint(1, 60)) # Add a small increment if needed


        # For intermediate stages, distribute the remaining time
        else:
             # Calculate remaining time and number of remaining transitions
             remaining_duration_seconds = (last_updated_date - current_timestamp).total_seconds()
             remaining_transitions = len(ordered_stages) - seq

             if remaining_transitions > 0:
                 # Estimate average time per remaining transition based on stage time ranges
                 # Need to consider the time ranges for the stages *between* the current stage and the final history stage (Negotiation or the stage before early loss)
                 # The stages for time range sum exclude the current stage and the very last stage (Won/Lost if applicable)
                 stages_for_time_range_sum = ordered_stages[seq:len(ordered_stages)-1]

                 estimated_min_remaining_days = sum(stage_time_ranges.get(s, (1, 1))[0] for s in stages_for_time_range_sum)
                 estimated_max_remaining_days = sum(stage_time_ranges.get(s, (1, 1))[1] for s in stages_for_time_range_sum)

                 # Add the time range for the next stage transition (from current stage to the next stage)
                 next_stage = ordered_stages[seq]
                 min_days_next, max_days_next = stage_time_ranges.get(next_stage, (1, 1))


                 # Determine the time to add for the current transition (from 'stage' to 'next_stage')
                 # Use a random value within a range around the estimated average, but bounded by stage time ranges
                 # The time range for the current transition is based on the *current* stage (which is stage)
                 min_days, max_days = stage_time_ranges.get(stage, (1, 1))
                 min_seconds = min_days * 24 * 3600
                 max_seconds = max_days * 24 * 3600

                 # Calculate a random increment, ensuring it's positive and within bounds relative to remaining time
                 # Avoid adding more time than is available until the last updated date
                 # Ensure there is enough time left for the minimum duration of the *remaining* stages (excluding the current transition)
                 min_seconds_for_remaining_stages = estimated_min_remaining_days * 24 * 3600
                 max_possible_increment_seconds = remaining_duration_seconds - min_seconds_for_remaining_stages # Ensure enough time for future min stages


                 # Ensure max_possible_increment_seconds is non-negative and at least 1 second if remaining_transitions > 0
                 max_possible_increment_seconds = max(1, max_possible_increment_seconds) if remaining_transitions > 0 else 0


                 # Select a random time to add, ensuring it is at least 1 second and does not exceed available time
                 # Also try to respect the stage_time_ranges for the current stage
                 # Ensure the lower bound is at least 1 second
                 lower_bound_seconds = max(min_seconds, 1)
                 upper_bound_seconds = min(max_seconds, max_possible_increment_seconds, remaining_duration_seconds)

                 # Ensure upper_bound_seconds is at least lower_bound_seconds
                 upper_bound_seconds = max(lower_bound_seconds, upper_bound_seconds)


                 time_to_add_seconds = random.uniform(lower_bound_seconds, upper_bound_seconds)


                 # Ensure the next timestamp is strictly after the current timestamp
                 next_timestamp = current_timestamp + timedelta(seconds=time_to_add_seconds)
                 while next_timestamp <= current_timestamp:
                      next_timestamp = current_timestamp + timedelta(seconds=random.uniform(1, 60)) # Add a small increment


                 timestamp = next_timestamp
             else:
                 # Should not happen for intermediate stages if logic is correct, but as a safeguard
                 timestamp = current_timestamp + timedelta(seconds=random.uniform(1, 60)) # Add a small increment


        stage_history_data.append({
            "History_ID": history_id,
            "Lead_ID": lead_id,
            "Stage": stage,
            "Timestamp": timestamp,
            "Rep_ID": lead.Rep_ID,
            "Stage_Sequence": seq,
            "Notes": f"Notes for stage {stage} for Lead {lead_id}"
        })
        history_id += 1
        current_timestamp = timestamp


df_stage_history = pd.DataFrame(stage_history_data)

df_stage_history['Timestamp'] = pd.to_datetime(df_stage_history['Timestamp'])
df_stage_history = df_stage_history.sort_values(by=['Lead_ID', 'Timestamp']).reset_index(drop=True)

stage_engagement_days = []

for lead_id, group in df_stage_history.groupby('Lead_ID'):
    group = group.sort_values(by='Timestamp').reset_index(drop=True)
    for i in range(len(group)):
        if i == 0:
            days = 0
        else:
            time_diff = group.iloc[i]['Timestamp'] - group.iloc[i-1]['Timestamp']
            days = round(time_diff.total_seconds() / (24 * 3600))
            days = max(days, 1)

        stage_engagement_days.append(days)

df_stage_history['Stage_Engagement (in Days)'] = stage_engagement_days

# Update df_lead's Current_Stage based on the latest stage in df_stage_history
latest_stage_indices = df_stage_history.groupby('Lead_ID')['Timestamp'].idxmax()
latest_stages = df_stage_history.loc[latest_stage_indices][['Lead_ID', 'Stage']]
latest_stages.rename(columns={'Stage': 'Actual_Current_Stage'}, inplace=True)

df_lead = df_lead.merge(latest_stages, on='Lead_ID', how='left', suffixes=('', '_latest_history'))
df_lead['Current_Stage'] = df_lead['Actual_Current_Stage'].fillna(df_lead['Current_Stage'])
df_lead = df_lead.drop(columns=['Actual_Current_Stage'])

display(df_lead.head())
display(df_lead.tail())
print(f"Generated {len(df_lead)} leads.")

display(df_stage_history.head())
display(df_stage_history.tail())
print(f"Generated {len(df_stage_history)} stage history entries.")

,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Last_Updated,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category
0,LDUBR-NA-001,Uber Lead,Uber,Outbound,NA,AS_NA,2025-03-24 11:49:55,Deal Lost,contact_34@uber.com,+1-0099765257,...,2025-05-09 09:57:27,Outbound,CMP_U,Low,7,6-12 months,50-100L,USA,Retail,Cloud
1,LDWMT-NA-002,Walmart Lead,Walmart,Outbound,NA,CL_NA,2025-06-02 21:03:44,Deal Won,contact_11@walmart.com,+1-7090059727,...,2025-07-22 20:06:49,Outbound,CMP_W,Medium,5,0-6 months,100L+,USA,Healthcare,Cloud
2,LDFRD-NA-003,Ford Motor Co Lead,Ford Motor Co,Outbound,NA,BJ_NA,2024-02-20 17:47:39,Deal Won,contact_83@fordmotorco.com,+1-2222484164,...,2024-04-22 00:11:43,Outbound,CMP_FMC,High,7,0-6 months,20-50L,USA,Retail,Data
3,LDAMZ-NA-004,Amazon Lead,Amazon,Inbound,NA,CL_NA,2023-10-09 19:32:02,Deal Won,contact_64@amazon.com,+1-5554699842,...,2023-11-16 01:57:55,Inbound,CMP_A,Medium,3,6-12 months,20-50L,USA,Finance,Data
4,LDMST-NA-005,Microsoft Lead,Microsoft,Outbound,NA,AS_NA,2024-06-12 22:45:26,Deal Lost,contact_27@microsoft.com,+1-0030706612,...,2024-07-03 03:48:00,Outbound,CMP_M,Low,3,0-6 months,100L+,USA,Healthcare,Cloud


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Last_Updated,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category
4995,LDFUTU-EU-4888,FutureSolutions Lead,FutureSolutions,Inbound,EU,EB_EU,2024-10-15 09:50:41,Deal Lost,contact_15@futuresolutions.com,+972-2130522820,...,2024-11-02 05:06:56,Inbound,CMP_F_4888,Medium,3,12-24 months,20-50L,Israel,Automotive,Consulting
4996,LDGLOB-LA-4889,GlobalSolutions Lead,GlobalSolutions,Outbound,LA,PY_LA,2025-06-10 11:27:29,Deal Won,contact_29@globalsolutions.com,+57-2783786847,...,2025-08-02 02:53:34,Outbound,CMP_G_4889,Medium,3,>24 months,20-50L,Colombia,Energy,Logistics
4997,LDCREA-APAC-4890,CreativeVentures Lead,CreativeVentures,Inbound,APAC,IG_APAC,2024-07-08 21:24:55,Deal Won,contact_93@creativeventures.com,+64-3599427694,...,2024-08-25 21:39:37,Inbound,CMP_C_4890,Medium,7,6-12 months,50-100L,New Zealand,Technology,AI
4998,LDCREA-APAC-4891,CreativeConsulting Lead,CreativeConsulting,Outbound,APAC,LS_APAC,2024-05-11 23:29:15,Deal Won,contact_47@creativeconsulting.com,+81-8662752026,...,2024-05-25 13:27:48,Outbound,CMP_C_4891,High,10,6-12 months,50-100L,Japan,Manufacturing,Staffing
4999,LDCREA-ME-4892,CreativeTech Lead,CreativeTech,Outbound,ME,NA_ME,2024-01-14 06:42:25,Deal Won,contact_45@creativetech.com,+966-2739863552,...,2024-01-19 10:15:25,Outbound,CMP_C_4892,High,3,12-24 months,50-100L,Saudi Arabia,Real Estate,Consulting


Generated 5000 leads.


,History_ID,Lead_ID,Stage,Timestamp,Rep_ID,Stage_Sequence,Notes,Stage_Engagement (in Days)
0,342,LDABB-EU-032,Lead Captured,2025-02-27 03:18:53,FG_EU,1,Notes for stage Lead Captured for Lead LDABB-E...,0
1,343,LDABB-EU-032,Qualified,2025-03-03 03:18:53,FG_EU,2,Notes for stage Qualified for Lead LDABB-EU-032,4
2,344,LDABB-EU-032,Contacted,2025-03-08 03:18:53,FG_EU,3,Notes for stage Contacted for Lead LDABB-EU-032,5
3,345,LDABB-EU-032,Responded,2025-03-11 03:18:53,FG_EU,4,Notes for stage Responded for Lead LDABB-EU-032,3
4,346,LDABB-EU-032,Demo Scheduled,2025-03-15 03:18:53,FG_EU,5,Notes for stage Demo Scheduled for Lead LDABB-...,4


,History_ID,Lead_ID,Stage,Timestamp,Rep_ID,Stage_Sequence,Notes,Stage_Engagement (in Days)
54995,953,LDZNR-APAC-087,Demo Completed,2023-09-13 10:22:19,JL_APAC,7,Notes for stage Demo Completed for Lead LDZNR-...,4
54996,954,LDZNR-APAC-087,Proposal Sent,2023-09-18 10:22:19,JL_APAC,8,Notes for stage Proposal Sent for Lead LDZNR-A...,5
54997,955,LDZNR-APAC-087,Proposal,2023-09-24 10:22:19,JL_APAC,9,Notes for stage Proposal for Lead LDZNR-APAC-087,6
54998,956,LDZNR-APAC-087,Negotiation,2023-10-01 10:22:19,JL_APAC,10,Notes for stage Negotiation for Lead LDZNR-APA...,7
54999,957,LDZNR-APAC-087,Deal Won,2023-10-01 10:23:01,JL_APAC,11,Notes for stage Deal Won for Lead LDZNR-APAC-087,1


Generated 55000 stage history entries.


In [22]:
# DEALS GENERATION
def generate_deals(df_lead, df_stage_history):
    deals_data = []
    deal_id_counter = 1

    # Define amount ranges for each budget tier (in Lakhs)
    # Assuming 'L' means Lakhs and '100L+' means 1 Crore or more
    budget_tier_amounts = {
        "20-50L": (20, 50),
        "50-100L": (50, 100),
        "100L+": (100, 500) # Assuming 1 Cr to 5 Cr for 100L+
    }

    closed_leads = df_lead[df_lead['Current_Stage'].isin(['Deal Won', 'Deal Lost'])].copy() # Create a copy to avoid SettingWithCopyWarning

    for index, lead in closed_leads.iterrows():
        lead_id = lead['Lead_ID']

        lead_captured_ts = df_stage_history[
            (df_stage_history['Lead_ID'] == lead_id) & (df_stage_history['Stage'] == 'Lead Captured')
        ]['Timestamp'].min()

        closed_ts = df_stage_history[
            (df_stage_history['Lead_ID'] == lead_id) & (df_stage_history['Stage'].isin(['Deal Won', 'Deal Lost']))
        ]['Timestamp'].max()

        if pd.notna(lead_captured_ts) and pd.notna(closed_ts):
            # Calculate total_duration_days for both 'Deal Won' and 'Deal Lost'
            total_duration_days = (closed_ts - lead_captured_ts).days


            # Determine the amount based on the Budget_Tier
            budget_tier = lead['Budget_Tier']
            amount_range = budget_tier_amounts.get(budget_tier, (1, 10)) # Default to small range if tier not found

            # Generate a random amount within the range, in Lakhs
            amount_in_lakhs = round(random.uniform(amount_range[0], amount_range[1]), 2)

            # Format the amount with 'Lakhs' or 'Crore'
            if amount_in_lakhs >= 100:
                 # Convert Lakhs to Crores for display
                 amount_display = f"{amount_in_lakhs / 100:.2f} Crore"
                 amount = amount_in_lakhs # Keep as number for calculations
            else:
                 amount_display = f"{amount_in_lakhs:.2f} Lakhs"
                 amount = amount_in_lakhs # Keep as number for calculations


            deals_data.append({
                "Deal_ID": f"DEAL_{deal_id_counter:04}",
                "Lead_ID": lead_id,
                "Company_ID": lead['Company_ID'],
                "Amount": amount_display, # Use the formatted amount for display
                "Status": lead['Current_Stage'],
                "Created_Date": lead_captured_ts.date().strftime("%Y-%m-%d"),
                "Closed_Date": closed_ts.date().strftime("%Y-%m-%d"),
                "Rep_ID": lead['Rep_ID'],
                "Total_Duration_Days": total_duration_days
            })
            deal_id_counter += 1

    return pd.DataFrame(deals_data)

df_deals = generate_deals(df_lead, df_stage_history)
display(df_deals.head())
print(f"Generated {len(df_deals)} deals.")

,Deal_ID,Lead_ID,Company_ID,Amount,Status,Created_Date,Closed_Date,Rep_ID,Total_Duration_Days
0,DEAL_0001,LDUBR-NA-001,CMP_U,84.06 Lakhs,Deal Lost,2025-03-24,2025-05-09,AS_NA,45
1,DEAL_0002,LDWMT-NA-002,CMP_W,2.71 Crore,Deal Won,2025-06-03,2025-07-22,CL_NA,49
2,DEAL_0003,LDFRD-NA-003,CMP_FMC,33.13 Lakhs,Deal Won,2024-02-21,2024-04-22,BJ_NA,60
3,DEAL_0004,LDAMZ-NA-004,CMP_A,49.29 Lakhs,Deal Won,2023-10-10,2023-11-19,CL_NA,40
4,DEAL_0005,LDMST-NA-005,CMP_M,3.08 Crore,Deal Lost,2024-06-13,2024-07-23,AS_NA,40


Generated 5000 deals.


In [23]:
!pip install --upgrade gspread gspread_dataframe

import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()


In [34]:
!pip install xlsxwriter

In [35]:
# Data Cleaning

import pandas as pd
import re

file_path = "/content/drive/MyDrive/CAPSTONE/CRM_Capstone.xlsx"

# Helper functions
def strip_strings(df):
    """Strip spaces from string columns and handle 'nan' as pd.NA"""
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].astype(str).str.strip().replace("nan", pd.NA)
    return df

def fix_email(email):
    """Validate and clean email"""
    if pd.isna(email):
        return email
    email = email.strip().lower()
    email = re.sub(r"[ ']", "", email)  # remove spaces and apostrophes
    pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    return email if re.match(pattern, email) else pd.NA

def safe_validate_foreign_key(df, fk_col, ref_df, ref_col):
    """Check for invalid foreign keys"""
    if fk_col not in df.columns or ref_col not in ref_df.columns:
        return pd.DataFrame()
    return df[~df[fk_col].isin(ref_df[ref_col])]

def clean_region_id(df):
    """Replace empty Region_ID with 'NA', preserve existing 'NA'"""
    region_cols = [c for c in df.columns if "Region_ID" in c]
    for col in region_cols:
        df[col] = df[col].replace("", "NA").replace(pd.NA, "NA")
    return df

def convert_dates(df, date_cols, to_date_only=False):
    """Convert columns to datetime/date"""
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")
            if to_date_only:
                df[col] = df[col].dt.date
    return df

# ---------- Load sheets ----------
sheets = ["Leads", "Sales_Reps", "Regions", "Companies", "Stage_History", "Deals"]
data = {sheet: pd.read_excel(file_path, sheet_name=sheet) for sheet in sheets}

# ---------- Cleaning ----------
# 1️⃣ Leads
leads = data["Leads"]
leads = strip_strings(leads)
leads = clean_region_id(leads)
leads.drop_duplicates(inplace=True)
if "Email" in leads.columns:
    leads["Email"] = leads["Email"].apply(fix_email)
# Convert any date columns
leads = convert_dates(leads, [c for c in leads.columns if "Date" in c])

# 2️⃣ Sales_Reps
sales_reps = data["Sales_Reps"]
sales_reps = strip_strings(sales_reps)
sales_reps = clean_region_id(sales_reps)
sales_reps.drop_duplicates(inplace=True)
if "Email" in sales_reps.columns:
    sales_reps["Email"] = sales_reps["Email"].apply(fix_email)
# Convert Join_Date to date only
sales_reps = convert_dates(sales_reps, ["Join_Date"], to_date_only=True)

# 3️⃣ Regions
regions = data["Regions"]
regions = strip_strings(regions)
regions = clean_region_id(regions)
regions.drop_duplicates(inplace=True)

# 4️⃣ Companies
companies = data["Companies"]
companies = strip_strings(companies)
companies = clean_region_id(companies)
companies.drop_duplicates(inplace=True)
invalid_regions = safe_validate_foreign_key(companies, "Region_ID", regions, "Region_ID")
if not invalid_regions.empty:
    print("⚠ Invalid Region IDs in Companies:\n", invalid_regions)

# 5️⃣ Stage_History
stage_history = data["Stage_History"]
stage_history = strip_strings(stage_history)
stage_history = clean_region_id(stage_history)
stage_history.drop_duplicates(inplace=True)
stage_history = convert_dates(stage_history, [c for c in stage_history.columns if "Date" in c])

# 6️⃣ Deals
deals = data["Deals"]
deals = strip_strings(deals)
deals = clean_region_id(deals)
deals.drop_duplicates(inplace=True)
# Created_Date and Closed_Date to date only
deals = convert_dates(deals, ["Created_Date", "Closed_Date"], to_date_only=True)

# FK checks
invalid_sales_rep = safe_validate_foreign_key(deals, "Sales_Rep_ID", sales_reps, "Sales_Rep_ID")
invalid_company = safe_validate_foreign_key(deals, "Company_ID", companies, "Company_ID")
if not invalid_sales_rep.empty:
    print("⚠ Invalid Sales Rep IDs in Deals:\n", invalid_sales_rep)
if not invalid_company.empty:
    print("⚠ Invalid Company IDs in Deals:\n", invalid_company)
invalid_deals_stage = safe_validate_foreign_key(stage_history, "Deal_ID", deals, "Deal_ID")
if not invalid_deals_stage.empty:
    print("⚠ Invalid Deal IDs in Stage_History:\n", invalid_deals_stage)

# ---------- Save cleaned Excel ----------
output_file = "/content/drive/MyDrive/CAPSTONE/Cleaned_CRM_Capstone.xlsx"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    leads.to_excel(writer, sheet_name="Leads", index=False)
    sales_reps.to_excel(writer, sheet_name="Sales_Reps", index=False)
    regions.to_excel(writer, sheet_name="Regions", index=False)
    companies.to_excel(writer, sheet_name="Companies", index=False)
    stage_history.to_excel(writer, sheet_name="Stage_History", index=False)
    deals.to_excel(writer, sheet_name="Deals", index=False)

print("✅ Cleaning complete. Saved as:", output_file)

✅ Cleaning complete. Saved as: /content/drive/MyDrive/CAPSTONE/Cleaned_CRM_Capstone.xlsx


In [24]:
# Use the spreadsheet name you created
spreadsheet = client.open("CRM_Capstone")

In [25]:
# Sheet mappings
sheet_mapping = {
    "Leads": df_lead,
    "Sales_Reps": df_sales_rep,
    "Regions": df_region,
    "Stage_History": df_stage_history,
    "Deals":df_deals,
    "Companies": df_company
}

# Write to each sheet
for sheet_name, df in sheet_mapping.items():
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=df.shape[0], cols=df.shape[1])

    worksheet.clear()
    set_with_dataframe(worksheet, df)

print("All DataFrames written to Google Sheets.")

All DataFrames written to Google Sheets.


In [26]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
